In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys

Using TensorFlow backend.


In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
smartphones = ["N5X"]
# TODO remove
participants = list(range(1, 4))
window_size = 20
constant_pixels = 0.06

# H5DF files contains 3 members: labels, segments, groups

In [4]:
filename = str(Path.home())+"/data/hdf/"+smartphones[0]+"-"+str(window_size)+".hdf"
f = h5py.File(filename, 'r')
# List all groups
#print("Keys: %s" % f.keys())
list(f.keys())
groups = f['fapra_imu']['groups'][()]
labels = f['fapra_imu']['labels'][()]
segments = f['fapra_imu']['segments'][()]

In [5]:
print(labels.shape)
print(segments.shape)
print(groups.shape)

(1446987, 2)
(1446987, 20, 18)
(1446987,)


In [6]:
## defining parameters for the input and network layers
## we are treating each segmeent or chunk as a 2D image (90 X 18)
numOfRows = segments.shape[1]
numOfColumns = segments.shape[2]

## reshaping the data for network input
reshapedSegments = segments.reshape(segments.shape[0], numOfRows, numOfColumns,1)

## Split Training- & Test-Data

In [7]:
logo = LeaveOneGroupOut()
logo.get_n_splits(segments, labels, groups)

17

In [8]:
## ConvLSTM net hyperparameters
numChannels = 1
numFilters = 128 # number of filters in Conv2D layer
# kernal size of the Conv2D layer
kernalSize1 = 1
# max pooling window size
poolingWindowSz = 2
# number of filters in fully connected layers
numNueronsFCL1 = 256
numNueronsFCL2 = 128
# number of epochs
Epochs = 10
# batchsize
batchSize = 200
# number of total clases
numClasses = labels.shape[1]
# dropout ratio for dropout layer
dropOutRatio = 0.2

In [ ]:
tf.get_default_graph()
for index, (train_index, test_index) in enumerate(logo.split(reshapedSegments, labels, groups)):

    # print('TRAIN:', train_index, 'TEST:', test_index)
    trainX, testX = reshapedSegments[train_index], reshapedSegments[test_index]
    trainY, testY = labels[train_index], labels[test_index]
    
    model = None
    model = Sequential()
    # adding the first convLSTM layer with 32 filters and 5 by 5 kernal size, using the rectifier as the activation function
    model.add(ConvLSTM2D(numFilters, (kernalSize1,kernalSize1),input_shape=(None, numOfRows, numOfColumns, 1),activation='relu', padding='same',return_sequences=True))

    ## adding a maxpooling layer
    model.add(TimeDistributed(MaxPooling2D(pool_size=(poolingWindowSz,poolingWindowSz),padding='valid')))

    ## adding a dropout layer for the regularization and avoiding over fitting
    model.add(Dropout(dropOutRatio))

    ## flattening the output in order to apple dense layer
    model.add(TimeDistributed(Flatten()))

    ## adding first fully connected layer with 256 outputs
    model.add(Dense(numNueronsFCL1, activation='relu'))

    ## adding second fully connected layer 128 outputs
    model.add(Dense(numNueronsFCL2, activation='relu'))

    ## flattening the output in order to apply the fully connected layer
    model.add(TimeDistributed(Flatten()))

    ## adding softmax layer for the classification
    model.add(Dense(numClasses)) #, activation='softmax'

    ## Compiling the model to generate a model
    def eucInMM(y_true, y_pred):
        return (K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))) * constant_pixels
    def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
    def euc(y_true, y_pred):
        return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))
    
    #optimizer = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #optimizer = optimizers.Adam(lr = 0.001, decay=1e-6)
    optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
    model.compile(loss=rmse, optimizer=optimizer, metrics=[eucInMM ,rmse, euc])

    model.summary()
    
    history = model.fit(np.expand_dims(trainX,1),np.expand_dims(trainY,1), 
                        validation_data=(np.expand_dims(testX,1),np.expand_dims(testY,1)),
                        epochs=9999,batch_size=batchSize,verbose=1)

    score = model.evaluate(np.expand_dims(testX,1),np.expand_dims(testY,1),verbose=2)
    print('%s: %.2f' % (model.metrics_names[1], score[1]))
    print('Baseline ConvLSTM Error: %.2f' %(1-score[1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_3 (ConvLSTM2D)  (None, None, 20, 18, 128) 66560     
_________________________________________________________________
time_distributed_7 (TimeDist (None, None, 10, 9, 128)  0         
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 10, 9, 128)  0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 11520)       0         
_________________________________________________________________
dense_7 (Dense)              (None, None, 256)         2949376   
_________________________________________________________________
dense_8 (Dense)              (None, None, 128)         32896     
_________________________________________________________________
time_distributed_9 (TimeDist (None, None, 128)         0         
__________

In [ ]:
10936 